# LSTM Text Classification

LSTM are a class of RNNs that are capable of learning long-term dependencies with feedback connections. In this project we use an LSTM model for classifying app review on a scale of 1 to 5.

This project focuses on text analytics which is the process of deriving high quality information from text.

Data link: https://www.kaggle.com/datasets/prakharrathi25/google-play-store-reviews

In [1]:
#Get required libraries
import re
import nltk
import time
import torch
import string
import numpy as np
import pandas as pd
import torch.nn as nn

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\uzmam\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
from nltk import PorterStemmer
import torch.nn.functional as F
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.utils import resample
from sklearn.model_selection import train_test_split


In [4]:
data=pd.read_csv('review.csv')
data.head()

reviewId            userName  \
0  gp:AOqpTOFxf3fttcT5DSvFIn9KPp5FErgH9yC533Fmoxv...      Donna Caritero   
1  gp:AOqpTOEq6rNIWLnPV4KFTctWvm0mpGEQljtD6mvy1H-...  Soumi Mukhopadhyay   
2  gp:AOqpTOE86hSyPRHZgYt28Uk5zGe4FZGb1hkmtFDiYJ2...   Theknown _unknown   
3  gp:AOqpTOHSuKkVTcM3QgCCKysHQlxEnk2ocOKsUMiMIJy...        Anthony Dean   
4  gp:AOqpTOEOrZt5H6jXPiplJyffCd5ZBnVXACTWgwNsF1R...   Neha Diana Wesley   

                                           userImage  \
0  https://play-lh.googleusercontent.com/a-/AOh14...   
1  https://play-lh.googleusercontent.com/a-/AOh14...   
2  https://play-lh.googleusercontent.com/a-/AOh14...   
3  https://play-lh.googleusercontent.com/a/AATXAJ...   
4  https://play-lh.googleusercontent.com/a-/AOh14...   

                                             content  score  thumbsUpCount  \
0  Overall it's really an amazing app. I've been ...      4            528   
1  Hey! Yes I gave a 5 star rating... coz I belie...      5            351   
2  Canva used to be a good app! But recently I've...      1            160   
3  It's a brilliant app, but I have just one prob...      5            145   
4  This was such a great app. I used to make BTS ...      4            142   

  reviewCreatedVersion                   at  \
0              2.116.0  2021-06-17 07:18:54   
1              2.116.0  2021-06-17 19:18:28   
2              2.116.0  2021-06-23 19:13:28   
3              2.116.0  2021-06-19 23:36:07   
4              2.116.0  2021-06-21 00:29:27   

                                        replyContent            repliedAt  
0  Hi Donna. We are sorry that your text or desig...  2021-06-19 21:24:32  
1                                                NaN                  NaN  
2  Hi there. We're sorry to hear that you are hav...  2021-06-26 20:20:56  
3                                                NaN                  NaN  
4  Hi Neha. Sorry to hear that you are unable to ...  2021-06-24 20:46:28

In [5]:
data.isna().sum()

reviewId                  0
userName                  0
userImage                 0
content                   0
score                     0
thumbsUpCount             0
reviewCreatedVersion      0
at                        0
replyContent            977
repliedAt               977
dtype: int64

In [6]:
data.describe()

score  thumbsUpCount
count  1500.000000    1500.000000
mean      4.154000       1.591333
std       1.361195      18.936791
min       1.000000       0.000000
25%       4.000000       0.000000
50%       5.000000       0.000000
75%       5.000000       0.000000
max       5.000000     528.000000

Most of the reply content is not available, so that is not a very useful feature. Our analysis will be based off of the content column (although other features can  be explored as well).

In [7]:
df=data[['content','score']]


content  score
0  Overall it's really an amazing app. I've been ...      4
1  Hey! Yes I gave a 5 star rating... coz I belie...      5
2  Canva used to be a good app! But recently I've...      1
3  It's a brilliant app, but I have just one prob...      5
4  This was such a great app. I used to make BTS ...      4

In [8]:
pd.set_option('display.max_colwidth', -1)

C:\Users\uzmam\AppData\Local\Temp/ipykernel_26100/3652881321.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [9]:
df.head()

content  \
0  Overall it's really an amazing app. I've been using this for the past 5 years however I only have one issue though and I wanted this to get address since I think this issue had lasted for how many years? The texts were blurred and when you zoom it out it's pixelated. I thought this issue only occurs on mobile apps however it was also present on the website. Please fix this. I still remember the time when I can export high definition texts and I love that experience. Thank you!   
1  Hey! Yes I gave a 5 star rating... coz I believe it deserves it! I mostly use the desktop version and I am seriously so satisfied with this app in both android and desktop version. I just came here to thank the developers for this beautiful app and its facilities. I literally find almost everything that I need for and the best part is even without the premium feature it provides us with a lot many facilities. I love it! Thank you..! ❤                                              
2  Canva used to be a good app! But recently I've been facing issues in saving my files. When I want my files to be saved in '.png' format, it gets saved in .jpg format! Moreover when I want my files to be saved it gets stuck at the downloading screen (where your so called magic does not happen) For now you'll be getting 1 stars from me. I hope you will fix the app soon! Thank you .                                                                                                      
3  It's a brilliant app, but I have just one problem, if I'm editing a post and I double tap the photo to reposition, if I then start to zoom the photo, 8 times out of 10 my app crashes and restarts, this is on my LG G5 SE, sometimes I've had to go over to the desktop version on my PC just to reposition and zoom the photo where I want it to then resume editing back on my mobile.                                                                                                          
4  This was such a great app. I used to make BTS memes through this. But after recent update... I'm not able to change my font style. The edit option of font is missing. I tried uninstalling and installing the app, but still it's isn't working. The edit option for font is appearing very rarely. The previous version was far better. Canva could you please work on it! 💜                                                                                                                      

   score  
0  4      
1  5      
2  1      
3  5      
4  4